In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import warnings
# filter warnings
warnings.filterwarnings('ignore')


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.preprocessing.image import load_img
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from tqdm import tqdm
from sklearn.utils import shuffle
from IPython.display import SVG
import matplotlib.image as imread
import cv2
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
path = "//kaggle/input/planet-understanding-the-amazon-from-space/"
os.listdir(path)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

['Kaggle-planet-train-tif.torrent',
 'test_v2_file_mapping.csv',
 'train_v2.csv',
 'Kaggle-planet-test-tif.torrent',
 'sample_submission_v2.csv']

In [9]:
# Load the Data

train_labels = pd.read_csv('//kaggle/input/planet-understanding-the-amazon-from-space/train_v2.csv/train_v2.csv')
test_labels = pd.read_csv('//kaggle/input/planet-understanding-the-amazon-from-space/sample_submission_v2.csv/sample_submission_v2.csv')
train_images = '//kaggle/input/planet-understanding-the-amazon-from-space/train-jpg'
test_images = '//kaggle/input/planet-understanding-the-amazon-from-space/test-jpg'


In [7]:
#View Shape of the Train Data

print("Train data: {}".format(train_labels.shape))

Train data: (40479, 2)


In [10]:
#View Shape of the Test Data

print("Test data: {}".format(test_labels.shape))

Test data: (61191, 2)
